In [1]:
import pandas as pd
import numpy as np
import psycopg2 as ps

In [2]:
df_ciudad = pd.read_csv('../data/ciudad.csv')
df_eventos = pd.read_csv("../data/eventos.csv", parse_dates=['fecha_inicio', 'fecha_fin'])
df_hoteles = pd.read_csv('../data/hoteles.csv')
df_clientes = pd.read_csv('../data/clientes.csv')
df_reservas = pd.read_csv("../data/reservas.csv", parse_dates=['fecha_reserva', 'inicio_estancia', 'final_estancia'])

In [3]:
df_eventos.duplicated().sum()

np.int64(0)

In [4]:
df_ciudad = df_ciudad.drop_duplicates()
df_hoteles = df_hoteles.drop_duplicates()
df_clientes = df_clientes.drop_duplicates()

In [5]:
df_ciudad.duplicated().sum()

np.int64(0)

In [6]:
conn = ps.connect(
    dbname="proyecto_etl",
    user="my_user",
    password="admin",
    host="localhost",
    port="5432"
)

In [7]:
cur = conn.cursor()

In [8]:
cur.execute("SELECT version();")
cur.fetchone()

('PostgreSQL 14.13 (Homebrew) on aarch64-apple-darwin23.4.0, compiled by Apple clang version 15.0.0 (clang-1500.3.9.4), 64-bit',)

In [9]:
# Convertir DataFrame a lista de tuplas
#records = df.to_records(index=False)
#records_list = list(records)

In [9]:
df_ciudad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ciudad     1 non-null      object
 1   id_ciudad  1 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 24.0+ bytes


In [11]:
d_insert_ciudad = ['Madrid']

In [12]:
insert_query = """
INSERT INTO ciudad (nombre_ciudad) VALUES (%s)
"""

In [13]:
cur.execute(insert_query, d_insert_ciudad)
conn.commit()

In [10]:
cur.execute('SELECT nombre_ciudad, id_ciudad FROM ciudad')
ciudades = dict(cur.fetchall())
ciudades

{'Madrid': 1}

In [11]:
df_eventos.head(1)

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion,id_ciudad
0,1,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,NaN,2025-02-24,2025-03-02,Biblioteca Pública Municipal Iván de Vargas (C...,1


In [16]:
d_insert_eventos = []

In [17]:
for i, row in df_eventos.iterrows():
    id_evento = row['id_evento']
    nombre_evento = row['nombre_evento']
    url_evento = row['url_evento']
    codigo_postal = row['codigo_postal'] if row['codigo_postal'] !=0 else None
    direccion = row['direccion'] if pd.notna(row['direccion']) else None
    horario = row['horario'] if pd.notna(row['horario']) else None
    fecha_inicio = row['fecha_inicio']
    fecha_fin = row['fecha_fin']
    organizacion = row['organizacion'] if pd.notna(row['organizacion']) else None
    id_ciudad = ciudades.get('Madrid')

    d_insert_eventos.append((id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))


In [ ]:
d_insert_eventos

In [19]:

query_eventos = """
    INSERT INTO eventos (id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
cur.executemany(query_eventos, d_insert_eventos)
conn.commit()

In [ ]:
df_hoteles = df_hoteles.rename(columns={"id_hotel_final": "id_hotel"})
df_hoteles['id_hotel'] = [(i + 1) for i in range(len(df_hoteles))]
df_hoteles

In [12]:
d_insert_hoteles = []

In [ ]:
for i, row in df_hoteles.iterrows():
    id_hotel = row['id_hotel']
    nombre_hotel = row['nombre_hotel']
    competencia = row['competencia']
    valoracion = row['valoracion']
    id_ciudad = ciudades.get('Madrid')

    d_insert_hoteles.append((id_hotel, nombre_hotel, competencia, valoracion, id_ciudad))

In [16]:

query_hoteles = """
    INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
    VALUES (%s, %s, %s, %s, %s)
"""

In [17]:
cur.executemany(query_hoteles, d_insert_hoteles)
conn.commit()

In [18]:
d_insert_clientes = []

In [19]:
for i, row in df_clientes.iterrows():
    id_cliente = row['id_cliente']
    nombre = row['nombre']
    apellido = row['apellido']
    mail = row['mail']

    d_insert_clientes.append((id_cliente, nombre, apellido, mail ))

In [20]:
query_clientes = """
    INSERT INTO clientes (id_cliente, nombre, apellido, mail)
    VALUES (%s, %s, %s, %s)
"""

In [21]:
cur.executemany(query_clientes, d_insert_clientes)
conn.commit()

In [12]:
df_reservas

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche,id_cliente,id_hotel
0,5256cc90-139b-43d2-8ec5-412495d751cf,2025-02-25,2025-03-01,2025-03-02,110.00,ID0,ID19
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,2025-02-25,2025-03-01,2025-03-02,172.00,ID1,ID20
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,2025-02-09,2025-03-01,2025-03-02,119.96,ID2,ID0
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,2025-02-08,2025-03-01,2025-03-02,211.53,ID3,ID1
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,2025-02-02,2025-03-01,2025-03-02,378.37,ID4,ID2
...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,2025-02-02,2025-03-01,2025-03-02,367.65,ID3319,ID7
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2025-02-07,2025-03-01,2025-03-02,144.35,ID12210,ID8
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,2025-02-08,2025-03-01,2025-03-02,463.32,ID7154,ID6
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,2025-02-10,2025-03-01,2025-03-02,247.63,ID13301,ID10


In [13]:
df_reservas['id_reserva'] = df_reservas['id_reserva'].str.replace('-', '', regex=False)

In [ ]:
df_reservas['id_hotel'] = df_reservas['id_hotel'].str.extract('(\d+)').astype(int)

In [21]:
df_reservas.head(30)

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche,id_cliente,id_hotel
0,5256cc90139b43d28ec5412495d751cf,2025-02-25,2025-03-01,2025-03-02,110.00,ID0,19
1,84fd6209bd8d4a92bfe6c68ee2c49271,2025-02-25,2025-03-01,2025-03-02,172.00,ID1,20
2,40c4cb55d1f5407b832f4756b8ff77b4,2025-02-09,2025-03-01,2025-03-02,119.96,ID2,0
3,f2ce8df5784443e18c0f97ce7a208f21,2025-02-08,2025-03-01,2025-03-02,211.53,ID3,1
4,57d4515a447d4067afcb9bdcf4e4e915,2025-02-02,2025-03-01,2025-03-02,378.37,ID4,2
5,ccbd9fe95a604f6fbed373a7158ecba7,2025-02-08,2025-03-01,2025-03-02,85.45,ID5,3
6,99c3dc4f663c45f7849aac9313f3746a,2025-02-09,2025-03-01,2025-03-02,309.80,ID6,4
7,4d65096e5f124ce79fd6dd20e3cf5194,2025-02-25,2025-03-01,2025-03-02,119.00,ID7,21
8,a5da09060fe54c30b193863df67a7b84,2025-02-10,2025-03-01,2025-03-02,434.88,ID8,5
9,458029adcc9b4d858290fec170bcc02d,2025-02-25,2025-03-01,2025-03-02,119.00,ID9,22


In [15]:
d_insert_reservas = []

In [22]:
hoteles_dict = dict(cur.fetchall())

ProgrammingError: no results to fetch

In [16]:
for i, row in df_reservas.iterrows():
    id_reserva = row['id_reserva']
    fecha_reserva = row['fecha_reserva']
    inicio_estancia = row['inicio_estancia']
    final_estancia = row['final_estancia']
    precio_noche = row['precio_noche']
    id_cliente = row['id_cliente']
    #id_hotel = row['id_hotel']
    id_hotel = hoteles_dict.get(nombre_hotel)

    d_insert_reservas.append((id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel))


In [17]:
query_reservas = """
    INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [18]:
cur.executemany(query_reservas, d_insert_reservas)
conn.commit()

ForeignKeyViolation: insert or update on table "reservas" violates foreign key constraint "reservas_id_hotel_fkey"
DETAIL:  Key (id_hotel)=(0) is not present in table "hoteles".


In [30]:
conn.rollback()

In [ ]:
cur.close()
conn.close()